In [1]:
import pandas as pd
import pypyodbc as odbc
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
"""
defining all the basic parameters
these are likely subject to change as the data collection methods change and grow
"""

DRIVER_NAME = 'SQL Server'
SERVER_NAME = 'PARTLABHMI'
DATABASE_NAME = 'IgnitionTest4'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trust_Connection =yes;
    uid=GradStudent;
    pwd=1000W/m2;
"""
conn = odbc.connect(connection_string)

In [3]:
"""
creating cursor to query data from the database
a cursor is an object that is stored and used to read over rows of data in a database
"""
lasel_cursor = conn.cursor()

In [4]:
"""
look at what tables exist and which ones may be of interest
here we just print the first ten; to view all tables in a database use print(tables)
"""
tables = []
for row in lasel_cursor.tables():
    tables.append(row['table_name'])
tables[0:10]

['sqlt_data_1_2023_08',
 'sqlt_data_1_2023_09',
 'sqlt_data_1_2023_10',
 'sqlt_data_1_2023_11',
 'sqlth_annotations',
 'sqlth_drv',
 'sqlth_partitions',
 'sqlth_sce',
 'sqlth_scinfo',
 'sqlth_te']

In [6]:
"""
sqlth_te is the table that has all of the tag id & descriptions
using these lines, we take the numerical tag ids and match them with their descriptions,
so later when we import a lot of datapoints we can easily rename the numerical tag ids with their description
"""
lasel_cursor.execute('SELECT * FROM dbo.sqlth_te')
tag_id = []
for row in lasel_cursor:
    tag_id.append(row)
tag_id = pd.DataFrame(tag_id, columns=['tag','description','scid','datatype','querymode','created','retired'])

In [7]:
"""
using f select we can name a table and pull that table from the database
the lines at the bottom replace those numerical tag ids with the str description
"""

table_name = 'dbo.sqlt_data_1_2023_09' #--------------------------> define the table here

lasel_cursor.execute(f"select * from {table_name}")
data = []
for row in lasel_cursor:
    data.append(row)
df = pd.DataFrame(data, columns=['tag','intvalue','floatvalue','stringvalue','datevalue','dataintegrity','TmStamp'])

df['description'] = np.nan
for tag in tag_id.tag.unique():
    df.loc[df['tag'] == tag, 'description'] = tag_id.loc[tag_id.tag == tag]['description'].to_string(index=False)

In [8]:
"""
the timestamp in the database is in milliseconds from midnight on January 1, 1970, so it needs to be converted
the timezone should also be specified
"""

df['TmStamp'] = pd.to_datetime(df.TmStamp, unit='ms')
df['TmStamp'] = df['TmStamp'] - timedelta(hours=5)
df.index = df['TmStamp']
df.index = df.index.tz_localize('America/Chicago', ambiguous='infer')

In [9]:
"""
let's preview what this table looks like, notice the tmstamps aren't evenly spaced and the columns are int/float/string/date
we need to pivot this so that the columns are all the different parameters shown in the description. each datetime should only have one row
that shows all parameters for that specific datetime
"""
df.head()

,tag,intvalue,floatvalue,stringvalue,datevalue,dataintegrity,TmStamp,description
TmStamp,,,,,,,,
2023-09-01 00:00:02.922000-05:00,1,NaN,0.1,None,None,192,2023-09-01 00:00:02.922,solisinverter1/dccurrent
2023-09-01 07:16:22.072000-05:00,1,NaN,0.2,None,None,192,2023-09-01 07:16:22.072,solisinverter1/dccurrent
2023-09-01 07:36:30.066000-05:00,1,NaN,0.4,None,None,192,2023-09-01 07:36:30.066,solisinverter1/dccurrent
2023-09-01 07:41:23.948000-05:00,1,NaN,0.6,None,None,192,2023-09-01 07:41:23.948,solisinverter1/dccurrent
2023-09-01 07:46:18.388000-05:00,1,NaN,0.4,None,None,192,2023-09-01 07:46:18.388,solisinverter1/dccurrent


In [10]:
"""
when we pivot, we need to specify the index, column names, and what value we want to keep
since some of these are integer and some are float, we will create two dfs for each
they remain seperate but can easily be joined using append, concat, merge, etc
"""

df_int = df.loc[df['intvalue'].notnull()]
df_int = df_int.pivot(index='TmStamp', columns='description', values='intvalue')

df_float = df.loc[df['floatvalue'].notnull()]
df_float = df_float.pivot(index='TmStamp', columns='description', values='floatvalue')

df_float_min = df_float.resample('T').mean()
df_float_min.index = df_float_min.index.tz_localize('America/Chicago')

In [11]:
"""
now if we print this df, we see the descriptions now each have their own column and there exists only one row for each specific date time
we also used a resampling function to get the data into minute intervals. resampling can also be used to get 5-min, 15-min, hour, etc
"""
df_float_min.head()

description,drakerreal/inv01_ac_energy,drakerreal/inv01_ac_energy_today,drakerreal/inv01_dc_power,drakerreal/inv02_ac_energy,drakerreal/inv02_ac_energy_today,drakerreal/inv02_dc_power,drakerreal/inv03_ac_energy,drakerreal/inv03_ac_energy_today,drakerreal/inv03_dc_power,drakerreal/inv04_ac_energy,...,sunny/grid freq,sunny/grid power,weatherstation/diffused,weatherstation/direct normal,weatherstation/globalirradiance,weatherstation/planeofarray,weatherstation/pressure,weatherstation/rain(mm),weatherstation/temperaturec,weatherstation/wind direction
TmStamp,,,,,,,,,,,,,,,,,,,,,
2023-09-01 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1013.139656,0.0,26.063864,123.543753
2023-09-01 00:01:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1013.131727,NaN,26.092498,129.341794
2023-09-01 00:02:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1013.127865,NaN,26.116671,131.980001
2023-09-01 00:03:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1013.137673,NaN,26.115328,110.976356
2023-09-01 00:04:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1013.148306,NaN,26.154463,100.685026


In [12]:
"""
if the data is going to be needed continually, converting it to a csv and storing locally may save time
reading from the sql database can sometimes take a while, depending on the length of the table 
if the file is large, it can be saved to different compressed types, like hdf or pkl
"""
df_float_min.to_csv(table_name+'_onemin.csv')
